In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
driver = webdriver.Chrome()
url = "https://ibotta.com/log-in"
driver.get(url)
# Log in then proceed. Automate at some point?

In [3]:
def get_num_pages():
    url = "https://ibotta.com/retailers/Publix/9?selectedCategoryIds=12%252C450%252C1%252C13%252C146%252C17%252C10%252C449%252C9%252C172%252C231%252C171%252C482%252C422%252C416%252C293%252C344%252C296"
    driver.get(url)
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "retailer-gallery"))
        )
    except:
        driver.quit()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    num_pages = (soup.find_all(class_="page-control"))[-2].get_text(strip=True)
    return num_pages


num_pages = get_num_pages()
print(num_pages)

6


In [4]:
# Get all URLs for Publix deals

def get_urls():
    urls = []
    for i in range(1, int(num_pages)+1):
        url = "https://ibotta.com/retailers/Publix/9?selectedCategoryIds=12%252C450%252C1%252C13%252C146%252C17%252C10%252C449%252C9%252C172%252C231%252C171%252C482%252C422%252C416%252C293%252C344%252C296"
        driver.get(url + f"&page={i}")
        try:
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "retailer-gallery"))
            )
        except:
            driver.quit()

        soup = BeautifulSoup(driver.page_source, 'lxml')
        cards = soup.find_all(class_="item-offer-card")
        temp_urls = ["https://ibotta.com" + card.find('a')['href'] for card in cards]
        urls.extend(temp_urls)
    
    return urls


urls = get_urls()
print(urls)
print(f"Number of deals: {len(urls)}")

['https://ibotta.com/offers/Coffee-Mate_Liquid_Creamer/1544848?categoryId=12', 'https://ibotta.com/offers/International_Delight_Cold_Foam_Creamer/1543435?categoryId=12', 'https://ibotta.com/offers/Coffee-Mate_Kelloggs_Eggo_or_Mean_Girls_Liquid_Creamer/1538163?categoryId=12', 'https://ibotta.com/offers/Light_%252B_Fit_Yogurt/1536562?categoryId=12', 'https://ibotta.com/offers/Activia_Probiotic_Low_Fat_Yogurt/1533103?categoryId=12', 'https://ibotta.com/offers/Vital_Farms_Pasture-Raised_Grade_A_Large_Eggs/1527157?categoryId=12', 'https://ibotta.com/offers/Danimals_Smoothies/1533765?categoryId=12', 'https://ibotta.com/offers/FAGE_BestSelf_Yogurt/1542020?categoryId=12', 'https://ibotta.com/offers/Pillsbury__Banana_Bread_Batter/1516691?categoryId=12', 'https://ibotta.com/offers/Velveeta_Cheese/1530351?categoryId=12', 'https://ibotta.com/offers/Yoplait_Go-GURT_or_Yoplait_Yogurt/1531917?categoryId=12', 'https://ibotta.com/offers/President_French-Style_Gourmet_Whipped_Creme_Topping/1531479?categ

In [5]:
# Iterate through all deals, grabbing data

def get_deals():
    df = pd.DataFrame(columns=["Title", "URL", "Offer", "Offer Restrictions", "Details"])
    for url in urls[:50]: # Change if you want all
        driver.get(url)
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "offer-spotlight"))
            )
        except:
            driver.quit()

        soup = BeautifulSoup(driver.page_source, 'lxml')
        box = soup.find(class_="offer-details__details-section")

        title = (soup.find(class_="offer-description__name-text"))
        offer = (soup.find(class_="offer-description__reward-amount-text"))
        offer_restrictions = (soup.find(class_="offer-details__restrictions-bold-long"))
        details = (box.find_all('p'))[-1]

        df.loc[-1] = [
            title.get_text(strip=True) if title else None,
            url,
            offer.get_text(strip=True) if offer else None,
            offer_restrictions.get_text(strip=True) if offer_restrictions else None,
            details.get_text() if details else None
            ]
        df.index = df.index + 1
        df.sort_index(inplace=True)
    
    driver.quit()
    return df


df = get_deals()
display(df)

,Title,URL,Offer,Offer Restrictions,Details
0,Fleischmann's RapidRise Instant Yeast,https://ibotta.com/offers/Fleischmanns_RapidRi...,$0.35 back,May be redeemed up to 5 times per receipt,Offer valid on Fleischmann's RapidRise Instant...
1,Del Monte Very Young Small Sweet Peas,https://ibotta.com/offers/Del_Monte_Very_Young...,$0.75 back,May be redeemed up to 5 times per receipt,Offer valid on Del Monte Very Young Small Swee...
2,Goodles Mac & Cheese,https://ibotta.com/offers/Goodles_Mac_and_Chee...,$0.50 back,May be redeemed up to 2 times per receipt,Offer valid on Goodles Mac & Cheese for any va...
3,Annie's™ Mac & Cheese,https://ibotta.com/offers/Annies_Mac_and_Chees...,$1.00 back,Must buy at least 4 to qualify,Offer valid when you buy 4 or more Annie's™ Ma...
4,Nature's Own® Soft White Keto Loaf,https://ibotta.com/offers/Natures_Own_Soft_Whi...,$1.75 back,May be redeemed up to 5 times per receipt,Offer valid on Nature's Own® Soft White Keto L...
5,Fresh Press Farms Extra Virgin Olive Oils,https://ibotta.com/offers/Fresh_Press_Farms_Ex...,$2.50 back,May be redeemed up to 3 times per receipt,Offer valid on Fresh Press Farms Extra Virgin ...
6,Pearls Olives to Go!,https://ibotta.com/offers/Pearls_Olives_to_Go!...,$0.50 back,May be redeemed up to 5 times per receipt,Offer valid on Pearls Olives to Go! for any va...
7,Royal Ready To Heat Rice,https://ibotta.com/offers/Royal_Ready_To_Heat_...,$1.00 back,May be redeemed up to 5 times per receipt,Offer valid on Royal Ready To Heat Rice for se...
8,Bumble Bee® Solid White Albacore Tuna Cans,https://ibotta.com/offers/Bumble_Bee_Solid_Whi...,$0.15 back,May be redeemed up to 2 times per receipt,Offer valid on Bumble Bee® Solid White Albacor...
9,Royal Ready To Heat Rice,https://ibotta.com/offers/Royal_Ready_To_Heat_...,$0.50 back,May be redeemed up to 5 times per receipt,Offer valid on Royal Ready To Heat Rice for se...
